In [1]:
import pandas as pd, numpy as np, re, datetime


In [5]:
from mezzanine.conf import settings


In [ ]:
register.settings().all()

## creating records from a df
First, create the dataframe from the mysql, save as csv with sep ~

In [5]:
mw = pd.read_csv('mw.csv', sep='~')
mw.rename(columns={'body':'content', 'dst':'slug'}, inplace=True)
mw['created'] = pd.to_datetime(mw.timestamp, utc=True, unit = 's')
mw = mw[['title','slug','created','content']]
mw.info()

In [6]:
mw['page_ptr_id'] = pd.Series(range(1,189))

In [23]:
mw.to_csv('mw.csv',sep='~',index=False)

In [3]:
mw = pd.read_csv('mw.csv', sep='~')

In [22]:
#search and replace
#mw.content.replace(to_replace='\n\r\n',value='</p><p>',inplace=True, regex=True)
#mw.content.replace(to_replace='\r</p>',value='</p>',inplace=True, regex=True)
mw.content.replace(to_replace='http://www.youtube',value='https://www.youtube',inplace=True, regex=True)
#print(repr(mw.loc[3,'content']))

## loading data from dataframe
Must load RichTextPage objects first as this is an inherited model. Cannot use bulk_create so must iterate.

This process also creates Page objects. 

Then we update the title, slug, in_menus, created, and publish_date fields in the Page objects.

In [24]:
for row in mw.itertuples():
    rtp = RichTextPage(content=row[4])
    rtp.save()
#this will take a while

In [25]:
#updating Page objects
for row in mw.itertuples():
    p= Page.objects.get(id=row[5])
    p.title = row[1]
    p.slug = row[2]
    p.created = row[3]
    p.publish_date = row[3]
    p.in_menus=[]
    p.save()


## Check the results

In [19]:
pages = pd.DataFrame(list(Page.objects.all().values()))
pages.head()

,_meta_title,_order,content_model,created,description,expiry_date,gen_description,id,in_menus,in_sitemap,...,login_required,parent_id,publish_date,short_url,site_id,slug,status,title,titles,updated
0,None,165,richtextpage,2011-01-25 14:51:54+00:00,\r\n\r\nAllan Savory\r\n\r\n\r\n\r\nManaging W...,None,True,166,[],True,...,False,None,2011-01-25 14:51:54+00:00,None,1,about,2,About ManagingWholes.com,About ManagingWholes.com,2018-01-15 10:29:28.478575+00:00
1,None,65,richtextpage,2010-02-10 16:16:01+00:00,by Peter Donovan\r\n\r\nThis essay was first p...,None,True,66,[],True,...,False,None,2010-02-10 16:16:01+00:00,None,1,leasing.htm,2,Achieving the potential of federal lands,Achieving the potential of federal lands,2018-01-15 10:29:27.718445+00:00
2,None,24,richtextpage,2010-02-05 16:19:06+00:00,by Peter Donovan (1995)\r,None,True,25,[],True,...,False,None,2010-02-05 16:19:06+00:00,None,1,southworth.htm,2,A conversation with Jack Southworth,A conversation with Jack Southworth,2018-01-15 10:29:27.408625+00:00
3,None,185,richtextpage,2013-03-09 16:02:35+00:00,\r,None,True,186,[],True,...,False,None,2013-03-09 16:02:35+00:00,None,1,Savory-TED,2,Allan Savory's TED talk,Allan Savory's TED talk,2018-01-15 10:29:28.607092+00:00
4,None,59,richtextpage,2010-02-09 03:42:09+00:00,by Peter Donovan\r\n\r\n\r\nPeter Donovan phot...,None,True,60,[],True,...,False,None,2010-02-09 03:42:09+00:00,None,1,merve.htm,2,A matter of method: Merve Wilkinson's Wildwood...,A matter of method: Merve Wilkinson's Wildwood...,2018-01-15 10:29:27.674256+00:00
